## ДЗ к Уроку 4.
### Непрерывные случайные величины. Функция распределения и плотность распределения вероятностей. Равномерное и нормальное распределение. Центральная предельная теорема

### 1. Задание:

_Случайная непрерывная величина A имеет равномерное распределение на промежутке (200, 800]._

_Найдите ее среднее значение и дисперсию._

In [1]:
a = 200
b = 800
M = (a + b) / 2
D = (b - a)**2 / 12
M, D

(500.0, 30000.0)

### 2. Задание:

_О случайной непрерывной равномерно распределенной величине B известно, что ее дисперсия равна 0.2._

_Можно ли найти правую границу величины B и ее среднее значение зная, что левая граница равна 0.5?_

_Если да, найдите ее._

In [2]:
D = 0.2
a = 0.5
b = (12 * D)**(1/2) + a
M = (a + b) / 2
b, M

(2.049193338482967, 1.2745966692414834)

### 3. Задание:

_Непрерывная случайная величина X распределена нормально и задана плотностью распределения_

## $$ f(x) = \frac{1}{4 \sqrt{2\pi}} e^{-\frac{(x+2)^2}{32}}$$

_Найдите:
а). M(X)
б). D(X)
в). std(X) (среднее квадратичное отклонение)_

Зная формулу распределения Гаусса:
## $$ f(x) = \frac{1}{\sigma \sqrt{2\pi}} e^{-\frac{(x-\mu)^2}{2\sigma^2}}$$
ищем в такой же формуле в условии задачи подставленные значения:

In [3]:
M = -2 # из сравнения (x + 2)**2 и (x - mu)**2 
D = 16 # из сравнения 32 и 2*sigma**2
S = 4 # из сравнения 4*(2*pi)**(1/2) и sigma*(2*pi)**(1/2)
M, D, S

(-2, 16, 4)

### 4. Задание:

_Рост взрослого населения города X имеет нормальное распределение._

_Причем, средний рост равен 174 см, а среднее квадратичное отклонение равно 8 см._

_Какова вероятность того, что случайным образом выбранный взрослый человек имеет рост:_
_а). больше 182 см
б). больше 190 см
в). от 166 см до 190 см
г). от 166 см до 182 см
д). от 158 см до 190 см
е). не выше 150 см или не ниже 190 см
ё). не выше 150 см или не ниже 198 см
ж). ниже 166 см._

In [4]:
import scipy.stats as ss

M = 174
S = 8

# а)
# вариант расчета веротяности через Z и стандартное нормальное распределение (закомментирован)
# Z = (182 - M) / S # = 1
# P = 1 - ss.norm.cdf(Z) # = 1 - 0.8413
# вариант расчета веротяности напрямую через нормальное распределение с параметрами M и S
P = 1 - ss.norm.cdf(182,M,S) # = 1 - 0.8413
print('а) P = ',P)

# б)
# вариант расчета веротяности через Z и стандартное нормальное распределение (закомментирован)
# Z = (190 - M) / S # = 2
# P = 1 - ss.norm.cdf(Z) # = 1 - 0.9772
# вариант расчета веротяности напрямую через нормальное распределение с параметрами M и S
P = 1 - ss.norm.cdf(190,M,S) # = 1 - 0.9772
print('б) P = ',P)

# в)
# вариант расчета веротяности через Z и стандартное нормальное распределение (закомментирован)
# Z_1 = (166 - M) / S # = -1
# Z_2 = (190 - M) / S # = 2
# P = ss.norm.cdf(Z_2) - ss.norm.cdf(Z_1) # = 0.9772 - 0.1587
# вариант расчета веротяности напрямую через нормальное распределение с параметрами M и S
P = ss.norm.cdf(190,M,S) - ss.norm.cdf(166,M,S) # = 0.9772 - 0.1587
print('в) P = ',P)

# г)
# вариант расчета веротяности через Z и стандартное нормальное распределение (закомментирован)
# Z_1 = (166 - M) / S # = -1
# Z_2 = (182 - M) / S # = 1
# P = ss.norm.cdf(Z_2) - ss.norm.cdf(Z_1) # = 0.8413 - 0.1587
# вариант расчета веротяности напрямую через нормальное распределение с параметрами M и S
P = ss.norm.cdf(182,M,S) - ss.norm.cdf(166,M,S) # = 0.8413 - 0.1587
print('г) P = ',P)

# д)
# вариант расчета веротяности через Z и стандартное нормальное распределение (закомментирован)
# Z_1 = (158 - M) / S # = -2
# Z_2 = (190 - M) / S # = 2
# P = ss.norm.cdf(Z_2) - ss.norm.cdf(Z_1) # = 0.9772 - 0.0228
# вариант расчета веротяности напрямую через нормальное распределение с параметрами M и S
P = ss.norm.cdf(190,M,S) - ss.norm.cdf(158,M,S) # = 0.9772 - 0.0228
print('д) P = ',P)

# е)
# вариант расчета веротяности через Z и стандартное нормальное распределение (закомментирован)
# Z_1 = (150 - M) / S # = -3
# Z_2 = (190 - M) / S # = 2
# P = 1 - ss.norm.cdf(Z_2) + ss.norm.cdf(Z_1) # = 1 - 0.9772 + 0.0013
# вариант расчета веротяности напрямую через нормальное распределение с параметрами M и S
P = 1 - ss.norm.cdf(190,M,S) + ss.norm.cdf(150,M,S) # = 1 - 0.9772 + 0.0013
print('е) P = ',P)

# ё)
# вариант расчета веротяности через Z и стандартное нормальное распределение (закомментирован)
# Z_1 = (150 - M) / S # = -3
# Z_2 = (198 - M) / S # = 3
# P = 1 - ss.norm.cdf(Z_2) + ss.norm.cdf(Z_1) # = 1 - 0.9987 + 0.0013
# вариант расчета веротяности напрямую через нормальное распределение с параметрами M и S
P = 1 - ss.norm.cdf(198,M,S) + ss.norm.cdf(150,M,S) # = 1 - 0.9987 + 0.0013
print('ё) P = ',P)

# ж)
# вариант расчета веротяности через Z и стандартное нормальное распределение (закомментирован)
# Z = (166 - M) / S # = -1
# P = ss.norm.cdf(Z) # = 0.1587
# вариант расчета веротяности напрямую через нормальное распределение с параметрами M и S
P = ss.norm.cdf(166,M,S) # = 0.1587
print('ж) P = ',P)


а) P =  0.15865525393145707
б) P =  0.02275013194817921
в) P =  0.8185946141203637
г) P =  0.6826894921370859
д) P =  0.9544997361036416
е) P =  0.0241000299798093
ё) P =  0.0026997960632601965
ж) P =  0.15865525393145707


### 5. Задание:

_На сколько сигм (средних квадратичных отклонений) отклоняется рост человека, равный 190 см, от 
математического ожидания роста в популяции, в которой M(X) = 178 см и D(X) = 25 кв.см?_

In [5]:
M = 178
D = 25
S = D**(1/2)
X = 190
Z = (X - M) / S
Z

2.4